In [98]:
import os
import re
import flywheel
from pprint import pprint as pp

# Assumes you are logged in via the CLI
fw = flywheel.Client()
print('Hello %s %s!' % (fw.get_current_user().firstname, fw.get_current_user().lastname))

Hello Michael Perry!


In [30]:
# Run MRQ gear on each session in the Descoteaux collection
collection = fw.collections.find('label=Descoteaux')[0]
sessions = collection.sessions()

# Get the gear we'll be running
gear = fw.lookup('gears/mrq')

# Modify the default configuration to account for the (non-standard) acquisition labels...
config = gear.get_default_config()
config['ir_regex'] = '[0-9].*_[0-9].*_IR.*'
config['spgr_regex'] = '[0-9].*_[0-9].*_SPGR.*'
config['session_split'] = ''

# Set an analysis label
ANALYSIS_LABEL = 'MRQ - %s' % (gear.gear.version)

# Find the session, which will be the destination
for s in sessions:
    print(s.label)
    # Run the gear/job, which returns the analysis ID
    analysis_id = gear.run(analysis_label=ANALYSIS_LABEL, config=config, destination=s)
    print(analysis_id)

9465
5cca1d9a7bb46c00364a80cf
9464
5cca1d9b7bb46c00364a80d1
8893
5cca1d9b7bb46c00364a80d3
8892
5cca1d9b7bb46c00354a3532
8763
5cca1d9b7bb46c00354a3534
8504
5cca1d9b7bb46c00354a3536
8214
5cca1d9b7bb46c00354a3538
5569
5cca1d9b7bb46c00364a80d5
5477
5cca1d9b7bb46c00364a80d7
5456
5cca1d9b7bb46c00364a80d9
5441
5cca1d9b7bb46c003249fdd5
5362
5cca1d9b7bb46c00364a80db
5312
5cca1d9b7bb46c00364a80dd
5245
5cca1d9b7bb46c00354a353a
5236
5cca1d9c7bb46c00364a80df
5228
5cca1d9c7bb46c00364a80e1
5214
5cca1d9c7bb46c00364a80e3
5180
5cca1d9c7bb46c00364a80e5
5139
5cca1d9c7bb46c00364a80e7
5138
5cca1d9c7bb46c00364a80e9
5131
5cca1d9c7bb46c00364a80eb
5129
5cca1d9c7bb46c00364a80ed
5109
5cca1d9c7bb46c00364a80ef
5107
5cca1d9c7bb46c00364a80f1
5084
5cca1d9c7bb46c00364a80f3
5082
5cca1d9c7bb46c00364a80f5
5073
5cca1d9c7bb46c00364a80f7
4996
5cca1d9c7bb46c00364a80f9
4885
5cca1d9c7bb46c00364a80fb
4881
5cca1d9d7bb46c00364a80fd
4853
5cca1d9d7bb46c00354a353c
4842
5cca1d9d7bb46c00354a353e
4808
5cca1d9d7bb46c00364a80ff
4775
5cca1

In [72]:
# Once the MRQ runs are done (verified in FW)

# Download the quantitative T1s
collection = fw.collections.find('label=Descoteaux')[0]

# Here we have to get the session in order to get the analyses
sessions = [ fw.get_session(x.id) for x in collection.sessions() ]

download_dir = '/scratch/purge/descoteaux'
if not os.path.isdir(download_dir):
    os.mkdir(download_dir)
    
# Regex to match for the t1 file
rex = re.compile('.*_.*-T1.nii.gz')

# For each session, find the analysis, and download the T1q data
for s in sessions:
    print(s.label)
    analysis = [ x for x in s.analyses if x.label.startswith('MRQ') ]
    if analysis:
        analysis = analysis[0]
        t1_file = [ x for x in analysis.files if rex.match(x.name) ]
        if t1_file:
            t1_file = t1_file[0]
            print(t1_file.name)
            dl_file = os.path.join(download_dir, t1_file.name.split('_')[1:][0])
            print(dl_file)
            t1_file.download(dl_file)
    else:
        print('%s has no matching analysis!' % s.label)
            


9465
s163_9465-T1.nii.gz
/scratch/purge/descoteaux/9465-T1.nii.gz
9464
s162_9464-T1.nii.gz
/scratch/purge/descoteaux/9464-T1.nii.gz
8893
s160_8893-T1.nii.gz
/scratch/purge/descoteaux/8893-T1.nii.gz
8892
s159_8892-T1.nii.gz
/scratch/purge/descoteaux/8892-T1.nii.gz
8763
s158_8763-T1.nii.gz
/scratch/purge/descoteaux/8763-T1.nii.gz
8504
8214
s156_8214-T1.nii.gz
/scratch/purge/descoteaux/8214-T1.nii.gz
5569
s148_5569-T1.nii.gz
/scratch/purge/descoteaux/5569-T1.nii.gz
5477
s147_5477-T1.nii.gz
/scratch/purge/descoteaux/5477-T1.nii.gz
5456
s146_5456-T1.nii.gz
/scratch/purge/descoteaux/5456-T1.nii.gz
5441
s145_5441-T1.nii.gz
/scratch/purge/descoteaux/5441-T1.nii.gz
5362
s144_5362-T1.nii.gz
/scratch/purge/descoteaux/5362-T1.nii.gz
5312
s143_5312-T1.nii.gz
/scratch/purge/descoteaux/5312-T1.nii.gz
5245
s077_5245-T1.nii.gz
/scratch/purge/descoteaux/5245-T1.nii.gz
5236
s141_5236-T1.nii.gz
/scratch/purge/descoteaux/5236-T1.nii.gz
5228
s140_5228-T1.nii.gz
/scratch/purge/descoteaux/5228-T1.nii.gz
5214


In [91]:
## Grab the T1w images from the zip file.

# This I decided to do after the fact, when I saw (https://groups.google.com/forum/#!topic/mrq-forum/bT5Msk119zE): 

import re

# Download the quantitative T1s and zip
collection = fw.collections.find('label=Descoteaux')[0]

# Have to get the session in order to get the analyses.
sessions = [ fw.get_session(x.id) for x in collection.sessions() ]

download_dir = '/scratch/purge/descoteaux'
if not os.path.isdir(download_dir):
    os.mkdir(download_dir)
    
# Regex to match for the t1 and zip files
rex = re.compile('.*T1w1.nii.gz')
ziprex = re.compile('.*-mrQ_Output.zip')

# For each session, find the analysis, output zip, and download the t1w data
for s in sessions:
    print(s.label)
    analysis = [ x for x in s.analyses if x.label.startswith('MRQ') ]
    if analysis:
        analysis = analysis[0]
        zip_file = [ x for x in analysis.files if ziprex.match(x.name) ]
        if zip_file: 
            zip_file = zip_file[0]
            zip_info = analysis.get_file_zip_info(zip_file.name)
            entry_name = [ x for x in zip_info.members if rex.match(x.path) ][0]
            print(entry_name.path)
            analysis.download_file_zip_member(zip_file.name, entry_name.path, os.path.join(download_dir, s.label + '-T1w.nii.gz'))
    else:
        print('%s has no matching analysis!' % s.label)

9465
s163_9465-mrQ_Output/OutPutFiles_1/SyntheticT1w/T1w1.nii.gz
9464
s162_9464-mrQ_Output/OutPutFiles_1/SyntheticT1w/T1w1.nii.gz
8893
s160_8893-mrQ_Output/OutPutFiles_1/SyntheticT1w/T1w1.nii.gz
8892
s159_8892-mrQ_Output/OutPutFiles_1/SyntheticT1w/T1w1.nii.gz
8763
s158_8763-mrQ_Output/OutPutFiles_1/SyntheticT1w/T1w1.nii.gz
8504
8214
s156_8214-mrQ_Output/OutPutFiles_1/SyntheticT1w/T1w1.nii.gz
5569
s148_5569-mrQ_Output/OutPutFiles_1/SyntheticT1w/T1w1.nii.gz
5477
s147_5477-mrQ_Output/OutPutFiles_1/SyntheticT1w/T1w1.nii.gz
5456
s146_5456-mrQ_Output/OutPutFiles_1/SyntheticT1w/T1w1.nii.gz
5441
s145_5441-mrQ_Output/OutPutFiles_1/SyntheticT1w/T1w1.nii.gz
5362
s144_5362-mrQ_Output/OutPutFiles_1/SyntheticT1w/T1w1.nii.gz
5312
s143_5312-mrQ_Output/OutPutFiles_1/SyntheticT1w/T1w1.nii.gz
5245
s077_5245-mrQ_Output/OutPutFiles_1/SyntheticT1w/T1w1.nii.gz
5236
s141_5236-mrQ_Output/OutPutFiles_1/SyntheticT1w/T1w1.nii.gz
5228
s140_5228-mrQ_Output/OutPutFiles_1/SyntheticT1w/T1w1.nii.gz
5214
s139_5214-mrQ_O

## NOTES

* 8504 & 4322 did not run through. I shared the SPGR coil-combined images for FA10, acpc-aligned.